<a href="https://colab.research.google.com/github/vishal7379/Colab/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip /content/archive.zip -d /content/

Archive:  /content/archive.zip
  inflating: /content/spider/README.txt  
  inflating: /content/spider/database/academic/academic.sqlite  
  inflating: /content/spider/database/academic/schema.sql  
  inflating: /content/spider/database/activity_1/activity_1.sqlite  
  inflating: /content/spider/database/activity_1/schema.sql  
  inflating: /content/spider/database/aircraft/aircraft.sqlite  
  inflating: /content/spider/database/aircraft/schema.sql  
  inflating: /content/spider/database/allergy_1/allergy_1.sqlite  
  inflating: /content/spider/database/allergy_1/schema.sql  
  inflating: /content/spider/database/apartment_rentals/apartment_rentals.sqlite  
  inflating: /content/spider/database/apartment_rentals/schema.sql  
  inflating: /content/spider/database/architecture/architecture.sqlite  
  inflating: /content/spider/database/architecture/schema.sql  
  inflating: /content/spider/database/assets_maintenance/assets_maintenance.sqlite  
  inflating: /content/spider/database/assets

In [8]:
# # ============================================================
# # SPIDER DATASET PREPROCESSING → SCHEMA-AWARE FORMAT
# # OUTPUT: spider_schema_aware.json (7000 examples)
# # ============================================================

# import json
# import re
# import os

# # ============================================================
# # 1️⃣ PATHS (DATASET ALREADY UNZIPPED)
# # ============================================================
# TRAIN_PATH = "/content/spider/train_spider.json"
# TABLES_PATH = "/content/spider/tables.json"

# assert os.path.exists(TRAIN_PATH), "❌ train_spider.json not found"
# assert os.path.exists(TABLES_PATH), "❌ tables.json not found"

# # ============================================================
# # 2️⃣ LOAD RAW SPIDER FILES
# # ============================================================
# with open(TRAIN_PATH, "r", encoding="utf-8") as f:
#     train_data = json.load(f)

# with open(TABLES_PATH, "r", encoding="utf-8") as f:
#     tables_data = json.load(f)

# print("✅ Loaded questions:", len(train_data))
# print("✅ Loaded schemas:", len(tables_data))

# # ============================================================
# # 3️⃣ BUILD DB → SCHEMA MAP
# # ============================================================
# db_schemas = {}

# for db in tables_data:
#     db_id = db["db_id"]
#     tables = db["table_names_original"]
#     columns = db["column_names_original"]

#     schema = {}
#     for tid, col in columns:
#         if tid == -1:
#             continue
#         table = tables[tid]
#         schema.setdefault(table, []).append(col)

#     db_schemas[db_id] = schema

# print("✅ Schema map built")

# # ============================================================
# # 4️⃣ TOKENIZE QUESTION
# # ============================================================
# def tokenize_question(text):
#     text = text.lower()
#     text = re.sub(r"[^a-z0-9_ ]", " ", text)
#     return text.split()

# # ============================================================
# # 5️⃣ FIND USED TABLES & COLUMNS FROM SQL
# # ============================================================
# def find_schema_mentions(sql, schema):
#     sql = sql.lower()
#     used_tables = set()
#     used_columns = set()

#     for table, cols in schema.items():
#         if re.search(rf"\b{re.escape(table.lower())}\b", sql):
#             used_tables.add(table)

#         for col in cols:
#             if re.search(rf"\b{re.escape(col.lower())}\b", sql):
#                 used_columns.add(f"{table}.{col}")

#     return used_tables, used_columns

# # ============================================================
# # 6️⃣ BUILD INPUT TOKENS + TOKEN TYPES
# # ============================================================
# def build_input_with_schema(question_tokens, schema):
#     tokens = []
#     token_types = []

#     # Question tokens → type 0
#     for tok in question_tokens:
#         tokens.append(tok)
#         token_types.append(0)

#     # Schema tokens
#     for table, cols in schema.items():
#         tokens.append(table)
#         token_types.append(1)
#         for col in cols:
#             tokens.append(col)
#             token_types.append(2)

#     return tokens, token_types

# # ============================================================
# # 7️⃣ BUILD SCHEMA LABELS
# # ============================================================
# def build_schema_labels(tokens, token_types, used_tables, used_columns):
#     labels = []

#     for tok, ttype in zip(tokens, token_types):
#         if ttype == 1:  # table
#             labels.append(1 if tok in used_tables else 0)
#         elif ttype == 2:  # column
#             labels.append(
#                 1 if any(tok == c.split(".")[1] for c in used_columns) else 0
#             )
#         else:
#             labels.append(0)

#     return labels

# # ============================================================
# # 8️⃣ CONVERT ONE EXAMPLE
# # ============================================================
# def convert_example(ex):
#     schema = db_schemas[ex["db_id"]]

#     q_tokens = tokenize_question(ex["question"])
#     used_tables, used_columns = find_schema_mentions(ex["query"], schema)

#     tokens, token_types = build_input_with_schema(q_tokens, schema)
#     schema_labels = build_schema_labels(tokens, token_types, used_tables, used_columns)

#     return {
#         "tokens": tokens,
#         "token_types": token_types,
#         "schema_labels": schema_labels,
#         "schema": schema,
#         "sql": ex["query"]
#     }

# # ============================================================
# # 9️⃣ PROCESS DATASET (LIMIT TO 7000)
# # ============================================================
# MAX_EXAMPLES = 7000
# processed = []

# for i, ex in enumerate(train_data[:MAX_EXAMPLES]):
#     processed.append(convert_example(ex))
#     if i > 0 and i % 500 == 0:
#         print(f"⏳ Processed {i} examples")

# print("✅ Total processed:", len(processed))

# # ============================================================
# # 🔟 SAVE OUTPUT
# # ============================================================
# OUTPUT_PATH = "/content/spider_schema_aware.json"

# with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
#     json.dump(processed, f, indent=2)

# print("✅ Saved to:", OUTPUT_PATH)

# # ============================================================
# # 1️⃣1️⃣ SANITY CHECK
# # ============================================================
# sample = processed[0]
# print("\n--- SAMPLE ---")
# print("Tokens:", sample["tokens"][:20])
# print("Token types:", sample["token_types"][:20])
# print("Schema labels:", sample["schema_labels"][:20])
# print("SQL:", sample["sql"])
import json, re, os

# Paths
TRAIN_PATH = "/content/spider/train_spider.json"
TABLES_PATH = "/content/spider/tables.json"

with open(TRAIN_PATH, "r") as f: train_data = json.load(f)
with open(TABLES_PATH, "r") as f: tables_data = json.load(f)

db_schemas = {}
for db in tables_data:
    schema = {}
    for tid, col in db["column_names_original"]:
        if tid == -1: continue
        table = db["table_names_original"][tid].lower()
        schema.setdefault(table, []).append(col.lower())
    db_schemas[db["db_id"]] = schema

def build_placeholders(schema):
    tmap = {t: f"T{i}" for i, t in enumerate(schema.keys())}
    cmap = {}
    cid = 0
    for t, cols in schema.items():
        for c in cols:
            cmap[f"{t}.{c}"] = f"C{cid}"
            cid += 1
    return tmap, cmap

def preprocess_sql(sql, tmap, cmap):
    sql = sql.lower()
    # Replace columns first (longer strings first to avoid partial matches)
    for k, v in sorted(cmap.items(), key=lambda x: -len(x[0])):
        sql = re.sub(rf"\b{re.escape(k)}\b", v, sql)
    for k, v in sorted(tmap.items(), key=lambda x: -len(x[0])):
        sql = re.sub(rf"\b{re.escape(k)}\b", v, sql)
    sql = re.sub(r"\b\d+\b", "VALUE", sql)
    return sql.upper().replace(",", " , ").replace("(", " ( ").replace(")", " ) ").split()

processed = []
for ex in train_data[:7000]:
    db_id = ex["db_id"]
    schema = db_schemas[db_id]
    tmap, cmap = build_placeholders(schema)

    # Question Tokens
    q_toks = re.sub(r"[^a-z0-9_ ]", " ", ex["question"].lower()).split()

    # Schema Tokens for Encoder
    s_toks, s_types = [], []
    for t, cols in schema.items():
        s_toks.append(t); s_types.append(1)
        for c in cols:
            s_toks.append(c); s_types.append(2)

    processed.append({
        "tokens": q_toks + s_toks,
        "token_types": [0]*len(q_toks) + s_types,
        "sql_tokens": preprocess_sql(ex["query"], tmap, cmap),
        "schema": schema,
        "db_id": db_id
    })

with open("spider_processed.json", "w") as f: json.dump(processed, f)

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import math
# import json
# import re
# from torch.utils.data import Dataset, DataLoader

# # ============================================================
# # 1️⃣ DEVICE
# # ============================================================
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Using device:", device)

# # ============================================================
# # 2️⃣ LOAD PREPROCESSED DATA (7000 examples)
# # ============================================================
# DATA_PATH = "spider_schema_aware.json"
# with open(DATA_PATH, "r", encoding="utf-8") as f:
#     data = json.load(f)

# data = data[:7000]
# print("Training examples:", len(data))

# # ============================================================
# # 3️⃣ BUILD ENCODER VOCAB
# # ============================================================
# def build_encoder_vocab(data):
#     vocab = {"<PAD>": 0, "<UNK>": 1}
#     idx = 2
#     for ex in data:
#         for tok in ex["tokens"]:
#             if tok not in vocab:
#                 vocab[tok] = idx
#                 idx += 1
#     return vocab

# encoder_vocab = build_encoder_vocab(data)
# enc_vocab_size = len(encoder_vocab)
# print("Encoder vocab size:", enc_vocab_size)

# # ============================================================
# # 4️⃣ DECODER SQL VOCAB (PLACEHOLDER-BASED)
# # ============================================================
# SQL_KEYWORDS = ["SELECT","FROM","WHERE","JOIN","ON","AS","DISTINCT"]
# LOGICAL_OPS = ["AND","OR","NOT"]
# COMPARISON_OPS = ["=","!=","<>",">","<",">=","<="]
# AGG_FUNCS = ["COUNT","SUM","AVG","MIN","MAX"]
# GROUPING = ["GROUP","BY","HAVING"]
# ORDERING = ["ORDER","ASC","DESC","LIMIT"]
# PUNCT = [",","(",")"]
# SPECIAL = ["<PAD>","<EOS>","VALUE"]

# MAX_TABLES = 128
# MAX_COLUMNS = 512
# TABLE_TOKENS = [f"T{i}" for i in range(MAX_TABLES)]
# COLUMN_TOKENS = [f"C{i}" for i in range(MAX_COLUMNS)]

# DECODER_VOCAB = (
#     SQL_KEYWORDS + LOGICAL_OPS + COMPARISON_OPS + AGG_FUNCS +
#     GROUPING + ORDERING + PUNCT + SPECIAL +
#     TABLE_TOKENS + COLUMN_TOKENS
# )

# token_to_id = {t:i for i,t in enumerate(DECODER_VOCAB)}
# id_to_token = {i:t for t,i in token_to_id.items()}
# VOCAB_SIZE = len(token_to_id)
# print("Decoder vocab size:", VOCAB_SIZE)

# # ============================================================
# # 5️⃣ SQL → PLACEHOLDER CONVERSION
# # ============================================================
# def build_schema_maps(schema):
#     tmap = {t: f"T{i}" for i,t in enumerate(schema.keys())}
#     cmap = {}
#     cid = 0
#     for t, cols in schema.items():
#         for c in cols:
#             cmap[f"{t}.{c}"] = f"C{cid}"
#             cid += 1
#     return tmap, cmap

# def sql_to_placeholder(sql, tmap, cmap):
#     sql = sql.lower()
#     for k,v in sorted(cmap.items(), key=lambda x:-len(x[0])):
#         sql = re.sub(rf"\b{re.escape(k)}\b", v, sql)
#     for k,v in tmap.items():
#         sql = re.sub(rf"\b{re.escape(k)}\b", v, sql)
#     sql = re.sub(r"\b\d+\b", "VALUE", sql)
#     return sql.upper()

# # ============================================================
# # 6️⃣ DATASET
# # ============================================================
# class SpiderDataset(Dataset):
#     def __init__(self, data, enc_vocab):
#         self.data = data
#         self.enc_vocab = enc_vocab

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         ex = self.data[idx]

#         x = [self.enc_vocab.get(t,1) for t in ex["tokens"]]
#         t = ex["token_types"]
#         s = ex["schema_labels"]

#         tmap, cmap = build_schema_maps(ex["schema"])
#         sql_ph = sql_to_placeholder(ex["sql"], tmap, cmap)
#         y = [token_to_id.get(tok, token_to_id["<PAD>"]) for tok in sql_ph.split()]
#         y.append(token_to_id["<EOS>"])

#         return (
#             torch.tensor(x),
#             torch.tensor(t),
#             torch.tensor(s, dtype=torch.float),
#             torch.tensor(y)
#         )

# def collate_fn(batch):
#     def pad(seqs, pad=0):
#         m = max(len(s) for s in seqs)
#         return torch.stack([
#             torch.cat([s, torch.full((m-len(s),), pad)]) for s in seqs
#         ])

#     x,t,s,y = zip(*batch)
#     return pad(x), pad(t), pad(s), pad(y)

# loader = DataLoader(
#     SpiderDataset(data, encoder_vocab),
#     batch_size=8,
#     shuffle=True,
#     collate_fn=collate_fn
# )

# # ============================================================
# # 7️⃣ TRANSFORMER MODULES
# # ============================================================
# class MultiHeadAttention(nn.Module):
#     def __init__(self, d, h):
#         super().__init__()
#         self.h = h
#         self.dk = d // h
#         self.qkv = nn.Linear(d, d*3)
#         self.o = nn.Linear(d, d)

#     def forward(self, q,k,v,mask=None):
#         B,T,D = q.size()
#         q,k,v = self.qkv(q).chunk(3,dim=-1)
#         q = q.view(B,T,self.h,self.dk).transpose(1,2)
#         k = k.view(B,-1,self.h,self.dk).transpose(1,2)
#         v = v.view(B,-1,self.h,self.dk).transpose(1,2)
#         scores = q @ k.transpose(-2,-1) / math.sqrt(self.dk)
#         if mask is not None:
#             scores = scores.masked_fill(mask==0, -1e9)
#         out = (scores.softmax(-1) @ v)
#         out = out.transpose(1,2).contiguous().view(B,T,D)
#         return self.o(out)

# class Encoder(nn.Module):
#     def __init__(self, vocab, d=256, h=8, ff=1024, layers=4):
#         super().__init__()
#         self.emb = nn.Embedding(vocab, d, padding_idx=0)
#         self.type_emb = nn.Embedding(3, d)
#         self.layers = nn.ModuleList([
#             nn.ModuleList([
#                 MultiHeadAttention(d,h),
#                 nn.Sequential(nn.Linear(d,ff), nn.ReLU(), nn.Linear(ff,d)),
#                 nn.LayerNorm(d),
#                 nn.LayerNorm(d)
#             ]) for _ in range(layers)
#         ])
#         self.schema_head = nn.Linear(d,1)

#     def forward(self, x, t, mask):
#         x = self.emb(x) + self.type_emb(t)
#         for attn, ff, n1, n2 in self.layers:
#             x = n1(x + attn(x,x,x,mask))
#             x = n2(x + ff(x))
#         return x, self.schema_head(x).squeeze(-1)

# class Decoder(nn.Module):
#     def __init__(self, vocab, d=256, h=8, ff=1024, layers=4):
#         super().__init__()
#         self.emb = nn.Embedding(vocab, d)
#         self.pos = nn.Embedding(512, d)
#         self.layers = nn.ModuleList([
#             nn.ModuleList([
#                 MultiHeadAttention(d,h),
#                 MultiHeadAttention(d,h),
#                 nn.Sequential(nn.Linear(d,ff), nn.ReLU(), nn.Linear(ff,d)),
#                 nn.LayerNorm(d),
#                 nn.LayerNorm(d),
#                 nn.LayerNorm(d)
#             ]) for _ in range(layers)
#         ])
#         self.out = nn.Linear(d, vocab)

#     def forward(self, y, enc, mask):
#         B,T = y.size()
#         x = self.emb(y) + self.pos(torch.arange(T, device=y.device))
#         for sa,ca,ff,n1,n2,n3 in self.layers:
#             x = n1(x + sa(x,x,x,mask))
#             x = n2(x + ca(x,enc,enc,None))
#             x = n3(x + ff(x))
#         return self.out(x)

# # ============================================================
# # 8️⃣ TRAINING
# # ============================================================
# encoder = Encoder(enc_vocab_size).to(device)
# decoder = Decoder(VOCAB_SIZE).to(device)

# opt = optim.Adam(
#     list(encoder.parameters()) + list(decoder.parameters()),
#     lr=3e-4
# )

# sql_loss_fn = nn.CrossEntropyLoss(ignore_index=token_to_id["<PAD>"])
# schema_loss_fn = nn.BCEWithLogitsLoss()

# EPOCHS = 8

# for ep in range(EPOCHS):
#     total = 0
#     for x,t,s,y in loader:
#         x,t,s,y = x.to(device), t.to(device), s.to(device), y.to(device)

#         enc_mask = (x!=0).unsqueeze(1).unsqueeze(2)
#         enc_out, sch = encoder(x,t,enc_mask)

#         dec_in, dec_tgt = y[:,:-1], y[:,1:]
#         T = dec_in.size(1)
#         causal = torch.tril(torch.ones(T,T,device=device)).unsqueeze(0).unsqueeze(0)

#         logits = decoder(dec_in, enc_out, causal)

#         loss = sql_loss_fn(logits.reshape(-1, VOCAB_SIZE), dec_tgt.reshape(-1))
#         loss += 0.3 * schema_loss_fn(sch*(t!=0), s*(t!=0))

#         opt.zero_grad()
#         loss.backward()
#         opt.step()

#         total += loss.item()

#     print(f"Epoch {ep+1} | Loss: {total/len(loader):.4f}")

# # ============================================================
# # 9️⃣ SAVE MODEL
# # ============================================================
# torch.save({
#     "encoder": encoder.state_dict(),
#     "decoder": decoder.state_dict(),
#     "encoder_vocab": encoder_vocab,
#     "decoder_vocab": token_to_id
# }, "nl2sql_schema_aware.pt")

# print("✅ Training complete & model saved")
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader

# --- Vocab ---
with open("spider_processed.json", "r") as f: data = json.load(f)

enc_vocab = {"<PAD>": 0, "<UNK>": 1}
for ex in data:
    for t in ex["tokens"]:
        if t not in enc_vocab: enc_vocab[t] = len(enc_vocab)

SQL_TOKENS = ["SELECT","FROM","WHERE","JOIN","ON","AND","GROUP","BY","ORDER","LIMIT","COUNT","SUM","AVG","MIN","MAX","=","<",">","!=","VALUE",",","(",")"]
dec_vocab = ["<PAD>", "<BOS>", "<EOS>"] + SQL_TOKENS + [f"T{i}" for i in range(10)] + [f"C{i}" for i in range(50)]
tok2id = {t: i for i, t in enumerate(dec_vocab)}
id2tok = {i: t for t, i in tok2id.items()}

class SpiderDataset(Dataset):
    def __init__(self, data): self.data = data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        ex = self.data[idx]
        x = [enc_vocab.get(t, 1) for t in ex["tokens"]]
        y = [tok2id["<BOS>"]] + [tok2id.get(t, 0) for t in ex["sql_tokens"]] + [tok2id["<EOS>"]]
        return torch.tensor(x), torch.tensor(ex["token_types"]), torch.tensor(y)

def collate(batch):
    x, t, y = zip(*batch)
    x = nn.utils.rnn.pad_sequence(x, batch_first=True)
    t = nn.utils.rnn.pad_sequence(t, batch_first=True)
    y = nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=0)
    return x, t, y

# --- Transformer Architecture ---
class TransformerSLM(nn.Module):
    def __init__(self, enc_size, dec_size, d=256):
        super().__init__()
        self.enc_emb = nn.Embedding(enc_size, d)
        self.type_emb = nn.Embedding(3, d)
        self.dec_emb = nn.Embedding(dec_size, d)
        self.tf = nn.Transformer(d_model=d, batch_first=True, nhead=8, num_encoder_layers=3, num_decoder_layers=3)
        self.fc = nn.Linear(d, dec_size)

    def forward(self, x, t, y):
        # Create masks
        y_mask = self.tf.generate_square_subsequent_mask(y.size(1)).to(y.device)
        x_emb = self.enc_emb(x) + self.type_emb(t)
        y_emb = self.dec_emb(y)
        out = self.tf(x_emb, y_emb, tgt_mask=y_mask)
        return self.fc(out)

# --- Training Loop ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerSLM(len(enc_vocab), len(dec_vocab)).to(device)
loader = DataLoader(SpiderDataset(data), batch_size=16, shuffle=True, collate_fn=collate)
opt = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss(ignore_index=0)



for epoch in range(8):
    total_loss = 0
    for x, t, y in loader:
        x, t, y = x.to(device), t.to(device), y.to(device)
        y_in, y_out = y[:, :-1], y[:, 1:]
        logits = model(x, t, y_in)
        loss = criterion(logits.reshape(-1, len(dec_vocab)), y_out.reshape(-1))
        opt.zero_grad(); loss.backward(); opt.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss/len(loader):.4f}")

torch.save({"model": model.state_dict(), "enc": enc_vocab, "dec": tok2id}, "slm_v1.pt")

Epoch 1 Loss: 1.0168


In [7]:
# # ============================================================
# # NL2SQL INFERENCE (FINAL – FIXED, NON-LOOPING)
# # ============================================================

# import torch
# import torch.nn as nn
# import math
# import re

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Using device:", device)

# # ---------------- LOAD MODEL ----------------
# ckpt = torch.load("nl2sql_schema_aware.pt", map_location=device)

# encoder_vocab = ckpt["encoder_vocab"]
# decoder_vocab = ckpt["decoder_vocab"]
# id_to_token = {v:k for k,v in decoder_vocab.items()}

# VOCAB_SIZE = len(decoder_vocab)

# # ---------------- MODEL DEFINITIONS ----------------
# class MultiHeadAttention(nn.Module):
#     def __init__(self, d, h):
#         super().__init__()
#         self.h = h
#         self.dk = d // h
#         self.qkv = nn.Linear(d, d*3)
#         self.o = nn.Linear(d, d)

#     def forward(self, q,k,v,mask=None):
#         B,T,D = q.size()
#         q,k,v = self.qkv(q).chunk(3,dim=-1)
#         q = q.view(B,T,self.h,self.dk).transpose(1,2)
#         k = k.view(B,-1,self.h,self.dk).transpose(1,2)
#         v = v.view(B,-1,self.h,self.dk).transpose(1,2)
#         scores = q @ k.transpose(-2,-1) / math.sqrt(self.dk)
#         if mask is not None:
#             scores = scores.masked_fill(mask==0, -1e9)
#         out = (scores.softmax(-1) @ v)
#         out = out.transpose(1,2).contiguous().view(B,T,D)
#         return self.o(out)

# class Encoder(nn.Module):
#     def __init__(self, vocab, d=256, h=8, ff=1024, layers=4):
#         super().__init__()
#         self.emb = nn.Embedding(vocab, d, padding_idx=0)
#         self.type_emb = nn.Embedding(3, d)
#         self.layers = nn.ModuleList([
#             nn.ModuleList([
#                 MultiHeadAttention(d,h),
#                 nn.Sequential(nn.Linear(d,ff), nn.ReLU(), nn.Linear(ff,d)),
#                 nn.LayerNorm(d),
#                 nn.LayerNorm(d)
#             ]) for _ in range(layers)
#         ])

#     def forward(self, x, t, mask):
#         x = self.emb(x) + self.type_emb(t)
#         for attn, ff, n1, n2 in self.layers:
#             x = n1(x + attn(x,x,x,mask))
#             x = n2(x + ff(x))
#         return x

# class Decoder(nn.Module):
#     def __init__(self, vocab, d=256, h=8, ff=1024, layers=4):
#         super().__init__()
#         self.emb = nn.Embedding(vocab, d)
#         self.pos = nn.Embedding(512, d)
#         self.layers = nn.ModuleList([
#             nn.ModuleList([
#                 MultiHeadAttention(d,h),
#                 MultiHeadAttention(d,h),
#                 nn.Sequential(nn.Linear(d,ff), nn.ReLU(), nn.Linear(ff,d)),
#                 nn.LayerNorm(d),
#                 nn.LayerNorm(d),
#                 nn.LayerNorm(d)
#             ]) for _ in range(layers)
#         ])
#         self.out = nn.Linear(d, vocab)

#     def forward(self, y, enc, mask):
#         B,T = y.size()
#         x = self.emb(y) + self.pos(torch.arange(T, device=y.device))
#         for sa,ca,ff,n1,n2,n3 in self.layers:
#             x = n1(x + sa(x,x,x,mask))
#             x = n2(x + ca(x,enc,enc,None))
#             x = n3(x + ff(x))
#         return self.out(x)

# encoder = Encoder(len(encoder_vocab)).to(device)
# decoder = Decoder(len(decoder_vocab)).to(device)

# encoder.load_state_dict(ckpt["encoder"], strict=False)
# decoder.load_state_dict(ckpt["decoder"])

# encoder.eval()
# decoder.eval()
# print("✅ Model loaded successfully")

# # ---------------- HELPERS ----------------
# def tokenize_question(text):
#     text = text.lower()
#     text = re.sub(r"[^a-z0-9_ ]", " ", text)
#     return text.split()

# def build_encoder_input(question, schema):
#     tokens, types = [], []
#     for tok in tokenize_question(question):
#         tokens.append(tok)
#         types.append(0)
#     for table, cols in schema.items():
#         tokens.append(table); types.append(1)
#         for col in cols:
#             tokens.append(col); types.append(2)

#     ids = [encoder_vocab.get(t, encoder_vocab["<UNK>"]) for t in tokens]
#     return (
#         torch.tensor(ids).unsqueeze(0).to(device),
#         torch.tensor(types).unsqueeze(0).to(device)
#     )

# # ---------------- SAFE GREEDY DECODING ----------------
# def generate_sql(question, schema, max_len=40):
#     x,t = build_encoder_input(question, schema)
#     mask = (x!=0).unsqueeze(1).unsqueeze(2)

#     with torch.no_grad():
#         enc = encoder(x,t,mask)

#     # Force SELECT only once
#     cur = torch.tensor([[decoder_vocab["SELECT"]]], device=device)
#     generated = ["SELECT"]

#     used = set(["SELECT"])

#     for _ in range(max_len):
#         T = cur.size(1)
#         causal = torch.tril(torch.ones(T,T,device=device)).unsqueeze(0).unsqueeze(0)

#         with torch.no_grad():
#             logits = decoder(cur, enc, causal)[0,-1]

#         # repetition penalty
#         for tok in used:
#             logits[decoder_vocab[tok]] -= 2.0

#         next_id = logits.argmax().item()
#         token = id_to_token[next_id]

#         if token == "<EOS>":
#             break

#         generated.append(token)
#         used.add(token)
#         cur = torch.cat([cur, torch.tensor([[next_id]], device=device)], dim=1)

#     return " ".join(generated)

# # ---------------- TEST ----------------
# schema = {
#      "department": [
#         "Department_ID",
#         "Name",
#         "Creation",
#         "Ranking",
#         "Budget_in_Billions",
#         "Num_Employees"
#       ],
#       "head": [
#         "head_ID",
#         "name",
#         "born_state",
#         "age"
#       ],
#       "management": [
#         "department_ID",
#         "head_ID",
#         "temporary_acting"
#       ]
# }

# question = "how many heads of the departments are older than 56"

# print("\n🟢 Generated SQL:")
# print(generate_sql(question, schema))
import torch

def generate_sql(question, schema, model_path="slm_v1.pt"):
    ckpt = torch.load(model_path)
    enc_v, dec_v = ckpt["enc"], ckpt["dec"]
    id2t = {i: t for t, i in dec_v.items()}

    model = TransformerSLM(len(enc_v), len(dec_v)).to(device)
    model.load_state_dict(ckpt["model"])
    model.eval()

    # Prepare input
    q_toks = question.lower().split()
    s_toks, s_types = [], []
    for t, cols in schema.items():
        s_toks.append(t); s_types.append(1)
        for c in cols: s_toks.append(c); s_types.append(2)

    x = torch.tensor([[enc_v.get(t, 1) for t in q_toks + s_toks]]).to(device)
    t = torch.tensor([[0]*len(q_toks) + s_types]).to(device)

    # Greedy Search with Repetition Penalty
    y = torch.tensor([[dec_v["<BOS>"]]]).to(device)
    for _ in range(30):
        with torch.no_grad():
            logits = model(x, t, y)[:, -1, :]

            # Repetition Penalty: reduce probability of already seen tokens
            for token_id in set(y[0].tolist()):
                logits[0, token_id] *= 0.8

            next_id = logits.argmax(dim=-1).item()
            if next_id == dec_v["<EOS>"]: break
            y = torch.cat([y, torch.tensor([[next_id]]).to(device)], dim=1)

    return " ".join([id2t[i] for i in y[0].tolist() if i not in [dec_v["<BOS>"], 0]])

# Test
schema_test = {"head": ["name", "age"], "department": ["id", "name"]}
print("\nGenerated SQL:", generate_sql("how many heads are there", schema_test))

Using device: cpu
✅ Model loaded successfully

🟢 Generated SQL:
SELECT T1 = T1
